In [4]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END, MessagesState
from app.chatbot.event_handler import create_event, get_events, update_event, delete_event
from langchain_core.prompts import ChatPromptTemplate
from langgraph.types import interrupt, Command
from langchain_core.messages import AIMessage, HumanMessage

# Load environment variables
load_dotenv()

tools = [create_event, get_events, update_event, delete_event]

# Initialize the model
def init_model() -> ChatOpenAI:
    try:
        MODEL_NAME = os.getenv("MODEL_NAME")
        OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(
            model=MODEL_NAME,
            temperature=0.3,
            api_key=OPENAI_API_KEY,
        )
        
        # llm.format = "json"
        print("Model initialized successfully:", llm)
        return llm
    except Exception as e:
        print(f"Model cannot be initialized: {e}")

llm = init_model()
    
def human_in_the_loop(state: dict):
    human = interrupt(state["messages"][-1].content)
    # information = "It is a personal meeting from 1 pm to 2pm and no attendees."
    state["messages"].append(HumanMessage(human))
    return state

# Define the agent node
def agent_node(state: dict):

    try:
        # Validate the state structure
        # if not isinstance(state, dict) or "context" not in state or "messages" not in state:
        #     raise ValueError("State must be a dictionary with 'context' and 'messages' keys")

        # llm = state["context"]["llm"]
        # tools = state["context"]["tools"]
        # if not llm or not tools:
        #     raise ValueError("LLM or tools missing from context")
        
        prompt = """
        You are a helpful assistant that can create, list, update, and delete google calendar events.
        You are in Eastern Standard Time Zone.
        Extract all the information from the user message and for information that is missing, ask the user for it causing least friction. Whenever you need data from the user, always ask the question starting with the phrase 'Human'."
        Assume data generously
        While updating or deleting events, get all the events for the mentioned date from 12am to 11:59pm. Use the id of that particular event to perform the necessary action."""
    
        llm = init_model()

        graph_agent = create_react_agent(llm, tools=tools, state_modifier=prompt)
        result = graph_agent.invoke(state)

        return result
    
    except Exception as e:
        print(f"Error in agent_node: {e}")
        return state

# Print stream function
def print_stream(stream):
    try:
        for s in stream:
            if isinstance(s, dict):
                # Handle "branch" condition
                if "branch" in s:
                    print(f"Branch condition met: {s['branch']}")
                # Handle messages from "agent"
                elif "agent" in s and "messages" in s["agent"]:
                    message = s["agent"]["messages"][-1]
                    if "AIMessage" in str(type(message)):  # Check if it's an AIMessage
                        print(message.content)
                else:
                    print("Other stream output:", s)
            else:
                print("Unexpected stream format:", s)
    except Exception as e:
        print(f"Error in print_stream: {e}")
        
# Main workflow

# Tools for handling events
tools = [create_event, get_events, update_event, delete_event]

# User input message
user_message = "Can you create an event on 26 January 2025 for a meeting at 110 Stewart Street?"
# user_message = "Can you list all the events I have on the 26 January 2025?"
# user_message = "Can you delete all the event on the 27 January 2025?"
# user_message = "Can you list all the events I have on the 27 January 2025? and then create an event on 27 jan 2025 from 5pm to 6pm for a meeting at 110 stewart street"
# user_message = "Can you delete the Meeting on 26th Jan 2025?"


# Initialize workflow state

initial_state = {
    "messages": [("user", user_message)],
}

def route(state: dict):
    if "Human" in state["messages"][-1].content:
        return "Human_Input"
    else:
        return END



# events = graph.stream(initial_state, config={"configurable": {"thread_id": "1"}})

# Print results
# print_stream(events)



# Resume using Command
# while "__interrupt__" in chunk:
#     for chunk in graph.stream(Command(resume=input("Enter additional info: ")), config=config):
#             print(chunk)

Model initialized successfully: client=<openai.resources.chat.completions.Completions object at 0x1189ce850> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x118c75c10> root_client=<openai.OpenAI object at 0x10f28ced0> root_async_client=<openai.AsyncOpenAI object at 0x118c74e50> model_name='gpt-4o-mini' temperature=0.3 model_kwargs={} openai_api_key=SecretStr('**********')


In [5]:
# Create the workflow
workflow = StateGraph(MessagesState)
workflow.add_node("agent", agent_node)
# workflow.add_node("Human_Input", human_in_the_loop)
workflow.add_edge(START, "agent")
# workflow.add_edge("Human_Input","agent")
# workflow.add_conditional_edges("agent", route)
workflow.add_edge("agent", END)

memory = MemorySaver()
# Compile and execute
graph = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

print(graph.get_state(config))

# Using stream() to directly surface the `__interrupt__` information.
for chunk in graph.stream(initial_state, config=config):
    print(chunk)

final_state = graph.get_state(config) 
for message in final_state.values["messages"]:
    print(message.content)



StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1'}}, metadata=None, created_at=None, parent_config=None, tasks=())
Model initialized successfully: client=<openai.resources.chat.completions.Completions object at 0x118c3e410> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x118cf7950> root_client=<openai.OpenAI object at 0x10f52d310> root_async_client=<openai.AsyncOpenAI object at 0x118cdbe90> model_name='gpt-4o-mini' temperature=0.3 model_kwargs={} openai_api_key=SecretStr('**********')
{'agent': {'messages': [HumanMessage(content='Can you create an event on 26 January 2025 for a meeting at 110 Stewart Street?', additional_kwargs={}, response_metadata={}, id='e64ab463-aa71-49fe-9573-e1a878dd3cb0'), AIMessage(content="Human, could you please provide me with the summary of the meeting, the start time, the end time, and any attendees' emails you'd like to include?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': 

In [20]:
final_state.values["messages"].append(HumanMessage("It is a personal meeting from 2pm to 3pm and no attendees."))

for chunk in graph.stream(final_state.values, config=config):
    print(chunk)

Model initialized successfully: client=<openai.resources.chat.completions.Completions object at 0x1179aa210> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1179a5f50> root_client=<openai.OpenAI object at 0x1179aa490> root_async_client=<openai.AsyncOpenAI object at 0x11795f510> model_name='gpt-4o-mini' temperature=0.3 model_kwargs={} openai_api_key=SecretStr('**********')
Event created: https://www.google.com/calendar/event?eid=ZnFqajJkdjRjbjVtZzV1MHA1bmc3NjU5M2tfMjAyNTAxMjZUMTkwMDAwWiByb2hpdHNoZWxrZTExQG0
{'agent': {'messages': [HumanMessage(content='Can you create an event on 26 January 2025 for a meeting at 110 Stewart Street?', additional_kwargs={}, response_metadata={}, id='8e5efd0a-330a-414a-9336-2eb7bbd87b62'), AIMessage(content='Human, could you please provide me with the summary of the meeting, the description, the start time, and the end time for the event on 26 January 2025? Additionally, if there are any attendees, please share their email addre

In [21]:
final_state = graph.get_state(config) 
for message in final_state.values["messages"]:
    print(message.content)

Can you create an event on 26 January 2025 for a meeting at 110 Stewart Street?
Human, could you please provide me with the summary of the meeting, the description, the start time, and the end time for the event on 26 January 2025? Additionally, if there are any attendees, please share their email addresses.
It is a personal meeting from 2pm to 3pm and no attendees.

Event created: https://www.google.com/calendar/event?eid=ZnFqajJkdjRjbjVtZzV1MHA1bmc3NjU5M2tfMjAyNTAxMjZUMTkwMDAwWiByb2hpdHNoZWxrZTExQG0
The personal meeting has been successfully created for 26 January 2025 from 2 PM to 3 PM at 110 Stewart Street. You can view the event [here](https://www.google.com/calendar/event?eid=ZnFqajJ2dHc3NjU5M2tfMjAyNTAxMjZUMTkwMDAwWiByb2hpdHNoZWxrZTExQG0).


In [ ]:
from openai import OpenAI
client = OpenAI()

audio_file= open("harvard.wav", "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file)

print(transcription.text)

The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.
